<a href="https://colab.research.google.com/github/joosk3R/jskRprac/blob/main/dacon_audio_classifier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet_pytorch
!pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=2837028749b644761ea1e724e4d3a88eda90f4c6412ef71fa8a8a89654c880ef
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to /tmp/pip-req-build-svvp7t_x
  Running command git clone --filter=blob:none --quiet https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git /tmp/pip-req-build-svvp7t_x
  Resolved https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to commit 9251dd7190eccfb1cd08e1ebce52e03cff0cb58a
  Preparing metadata (setup.py) ... done
  Created wheel for tor

In [ ]:
%cd /content/open

/content/open


In [ ]:
!unzip -qq "open (5).zip"

In [ ]:
import datetime
import warnings
warnings.filterwarnings('ignore')
import torch
from glob import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from torch_poly_lr_decay import PolynomialLRDecay
import random
from torchvision import models
from sklearn.metrics import accuracy_score, log_loss
import math
import librosa


os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'
torch.set_num_threads(8)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
test_x = pd.read_csv("test.csv")
africa_train_paths = sorted(glob("train/*.wav"))
test_paths = test_x["path"]

# https://dacon.io/competitions/official/235738/codeshare/2688
def load_data(paths):
    result = []
    for path in tqdm(paths):
        # sr = 16000이 의미하는 것은 1초당 16000개의 데이터를 샘플링 한다는 것입니다.
        data, sr = librosa.load(path, sr = 16000)
        result.append(data)
    result = np.array(result)
    # 메모리가 부족할 때는 데이터 타입을 변경해 주세요 ex) np.array(data, dtype = np.float32)

    return result




def get_feature(data, sr = 16000, n_fft = 2048, win_length = 200, hop_length = 160, n_mels = 64):
    mel = []
    for i in tqdm(data):
        # win_length 는 음성을 작은 조각으로 자를때 작은 조각의 크기입니다.
        # hop_length 는 음성을 작은 조각으로 자를때 자르는 간격을 의미합니다.
        # n_mels 는 적용할 mel filter의 개수입니다.
        mel_ = librosa.feature.melspectrogram(y=i, sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)

#     mel_mean = mel.mean()
#     mel_std = mel.std()
#     mel = (mel - mel_mean) / mel_std

    return mel


def set_length(data, d_mini):

    result = []
    for value in tqdm(data):
        value = value[:d_mini]
        if len(value)<d_mini:
            value = np.append(value, [0]*(d_mini-len(value)))
        result.append(value)
    result = np.array(result)

    return result

In [ ]:
africa_train_data = load_data(africa_train_paths)
np.save("/content/drive/MyDrive/gicback/africa-sorted.npy", africa_train_data)
train_data_list = [africa_train_data]




test_data = load_data(test_paths)
np.save("/content/drive/MyDrive/gicback/test_npy.npy", test_data)

# africa_train_data = np.load("./dataset/africa-sorted.npy", allow_pickle = True)
# australia_train_data = np.load("./dataset/australia-sorted.npy", allow_pickle = True)
# canada_train_data = np.load("./dataset/canada-sorted.npy", allow_pickle = True)
# england_train_data = np.load("./dataset/england-sorted.npy", allow_pickle = True)
# hongkong_train_data = np.load("./dataset/hongkong-sorted.npy", allow_pickle = True)
# us_train_data = np.load("./dataset/us-sorted.npy", allow_pickle = True)
# train_data_list = [africa_train_data, australia_train_data, canada_train_data, england_train_data, hongkong_train_data, us_train_data]

100%|██████████| 1881/1881 [00:00<00:00, 2128.63it/s]


In [ ]:
train_x = np.concatenate(train_data_list, axis=0)
train_x = set_length(train_x, 64000)

train_x_200 = get_feature(data = train_x, win_length=200)
train_x_400 = get_feature(data = train_x, win_length=400)
train_x_800 = get_feature(data = train_x, win_length=800)
train_x_1000 = get_feature(data = train_x, win_length=1000)


train_x_200 = train_x_200.reshape(train_x_200.shape[0], train_x_200.shape[1], train_x_200.shape[2], 1)
train_x_400 = train_x_400.reshape(train_x_400.shape[0], train_x_400.shape[1], train_x_400.shape[2], 1)
train_x_800 = train_x_800.reshape(train_x_800.shape[0], train_x_800.shape[1], train_x_800.shape[2], 1)
train_x_1000 = train_x_1000.reshape(train_x_1000.shape[0], train_x_1000.shape[1], train_x_1000.shape[2], 1)


train_x_multi = np.concatenate([train_x_200,
                                train_x_400,
                                train_x_800], -1)
np.save('/content/drive/MyDrive/gicback/train_x_multi.npy', train_x_multi)

100%|██████████| 5001/5001 [01:24<00:00, 59.07it/s]


In [ ]:
test_x = np.load('/content/drive/MyDrive/gicback/test_npy.npy', allow_pickle=True)
test_x = set_length(test_x, 64000)
test_x_200 = get_feature(data = test_x, win_length=200)
test_x_400 = get_feature(data = test_x, win_length=400)
test_x_800 = get_feature(data = test_x, win_length=800)
test_x_1000 = get_feature(data = test_x, win_length=1000)

test_x_200 = test_x_200.reshape(test_x_200.shape[0], test_x_200.shape[1], test_x_200.shape[2], 1)
test_x_400 = test_x_400.reshape(test_x_400.shape[0], test_x_400.shape[1], test_x_400.shape[2], 1)
test_x_800 = test_x_800.reshape(test_x_800.shape[0], test_x_800.shape[1], test_x_800.shape[2], 1)
test_x_1000 = test_x_1000.reshape(test_x_1000.shape[0], test_x_1000.shape[1], test_x_1000.shape[2], 1)

test_x_multi = np.concatenate([test_x_200,
                                test_x_400,
                                test_x_800], -1)
np.save('/content/drive/MyDrive/gicback/test_x_multi.npy', test_x_multi)

100%|██████████| 1881/1881 [00:32<00:00, 57.72it/s]


In [ ]:
xxx= pd.read_csv("/content/open/train.csv")
train_y = np.array(xxx["label"])
np.save('/content/drive/MyDrive/gicback/train_y_sort.npy', train_y)

In [ ]:
train_y

array([1, 2, 4, ..., 1, 1, 4])

In [ ]:
class conv_bn_relu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(conv_bn_relu, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        self.BN = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.conv(x)
        x = self.BN(x)
        x = self.relu(x)
        return x


class Network(nn.Module):
    def __init__(self, N):
        super(Network, self).__init__()
        self.N = N
        self.AveragePooling = nn.AvgPool2d(2)
        self.MaxPooling = nn.MaxPool2d(2)


        self.input_conv = conv_bn_relu(in_channels=3, out_channels=self.N, kernel_size=3)

        self.block1 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )

        self.block2 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )

        self.block3 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )

        self.block4 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )

        self.pool_block = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AdaptiveAvgPool2d(1)
        )

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=self.N*2, out_features=6),
        )

    def forward(self, x, out=''):
        x = self.input_conv(x)
        x = self.MaxPooling(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool_block(x)
        x=self.output(x)

        if out=='sigmoid':
            x = F.sigmoid(x)
        return x


class VoiceDatasetSimple(Dataset):
        def __init__(self, X, y, transform, inference=False, roll=False):
            self.X = X
            self.y = y
            self.transform = transform
            self.inference = inference
            self.roll = roll
        def __len__(self):
            return len(self.X)

        def __getitem__(self, idx):
            X = self.X[idx]
            X = (X-train_x_min)/(train_x_max-train_x_min)

            if self.inference:
                X = self.transform(X)
                return X
            else:
                if (self.roll==True) and (random.randint(0, 1)==1):
                    X = np.roll(X,random.randint(-200, 200), axis=1)

                X = self.transform(X)
                y = self.y[idx]

                onehot = np.zeros(6)
                onehot[y] = 1.
                y = onehot
                return X, y

def model_save(model, path):
    torch.save({
        'model': model.state_dict(),
    }, path)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    device_ids = list(range(torch.cuda.device_count()))
else:
    device_ids = None

In [ ]:
train_x = np.load('/content/drive/MyDrive/gicback/train_x_multi.npy')
idx = [k+person for k in range(0, len(train_x), 3)][:-1]

In [ ]:
train_x[idx].shape

In [ ]:
for person in range(3):
    train_x = np.load('/content/drive/MyDrive/gicback/train_x_multi.npy')
    train_y = np.load('/content/drive/MyDrive/gicback/train_y_sort.npy')
    train_x_min = train_x.min()
    train_x_max = train_x.max()


    idx = [k+person for k in range(0, len(train_x), 3)][:-1]
    train_x=train_x[idx]
    train_y=train_y[idx]


    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    folds=[]
    for train_idx, valid_idx in skf.split(train_x, train_y):
        folds.append((train_idx, valid_idx))


    for fold in range(5):
        epochs=45
        batch_size=128
        model_name = f'network-epoch{epochs}-person({person})-fold({fold}).pth'
        train_idx, valid_idx = folds[fold]

        transform = transforms.Compose([
            transforms.ToTensor(),
        ])

        train_dataset = VoiceDatasetSimple(X=train_x[train_idx], y=train_y[train_idx], transform=transform, roll=False)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

        valid_dataset = VoiceDatasetSimple(X=train_x[valid_idx], y=train_y[valid_idx], transform=transform)
        valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


        # model compile
        model = Network(16).to(device)
        model = nn.DataParallel(model, device_ids=device_ids)


        # optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr =1e-3)
        Q = math.floor(len(train_dataset)/batch_size+1)*epochs/7
        lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = Q)

        # loss
        criterion = nn.CrossEntropyLoss()


        best = 9999
        for epoch in range(epochs):
            start = time.time()
            model.train()
            train_loss=0
            train_pred_list=[]
            train_true_list=[]
            train_log_loss=0
            for X, y in (train_loader):
                X = torch.tensor(X, dtype=torch.float32, device=device)
                y = torch.tensor(y, dtype=torch.float32 , device=device)

                optimizer.zero_grad()
                pred = model(X)
                loss = criterion(pred, y.argmax(1))
                loss.backward()
                optimizer.step()
                lrs.step()

                train_pred_list += pred.argmax(1).detach().cpu().numpy().tolist()
                train_true_list += y.argmax(1).detach().cpu().numpy().tolist()
                train_loss+=loss.item()
            train_accuracy=accuracy_score(train_true_list, train_pred_list)



            with torch.no_grad():
                model.eval()
                valid_loss=0
                valid_log_loss=0
                valid_pred_list=[]
                valid_true_list=[]
                for X, y in (valid_loader):
                    X = torch.tensor(X, dtype=torch.float32, device=device)
                    y = torch.tensor(y, dtype=torch.float32 , device=device)

                    pred = model(X)
                    loss = criterion(pred, y.argmax(1))

                    valid_pred_list += pred.argmax(1).detach().cpu().numpy().tolist()
                    valid_true_list += y.argmax(1).detach().cpu().numpy().tolist()
                    valid_loss+=loss.item()

            valid_accuracy=accuracy_score(valid_true_list, valid_pred_list)

            if valid_loss/len(valid_loader) < best:
                model_save(model, f'{model_name}')
                best = valid_loss/len(valid_loader)

            print(f'===================== Epoch : {epoch+1}/{epochs}    time : {time.time()-start:.0f}s =====================')
            print(f'TRAIN -> loss : {train_loss/len(train_loader):.5f}     accuracy : {train_accuracy:.5f}')
            print(f'VALID -> loss : {valid_loss/len(valid_loader):.5f}     accuracy : {valid_accuracy:.5f}    best : {best:.5f}\n\n')

===================== Epoch : 1/45    time : 4s =====================
TRAIN -> loss : 1.70721     accuracy : 0.27402
VALID -> loss : 1.80268     accuracy : 0.17066    best : 1.80268


===================== Epoch : 2/45    time : 4s =====================
TRAIN -> loss : 1.61283     accuracy : 0.35210
VALID -> loss : 1.83551     accuracy : 0.17066    best : 1.80268


===================== Epoch : 3/45    time : 4s =====================
TRAIN -> loss : 1.56638     accuracy : 0.38138
VALID -> loss : 1.85894     accuracy : 0.11377    best : 1.80268


===================== Epoch : 4/45    time : 4s =====================
TRAIN -> loss : 1.53550     accuracy : 0.39715
VALID -> loss : 1.70314     accuracy : 0.28443    best : 1.70314


===================== Epoch : 5/45    time : 4s =====================
TRAIN -> loss : 1.51195     accuracy : 0.40766
VALID -> loss : 1.67789     accuracy : 0.31737    best : 1.67789


===================== Epoch : 6/45    time : 4s =====================
TRAIN -> l

Exception ignored in: <function _after_fork at 0x7fa01d259990>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1622, in _after_fork
    threads.update(_dangling)
  File "/usr/lib/python3.10/_weakrefset.py", line 66, in __iter__
    item = itemref()
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
X_test = np.load('/content/drive/MyDrive/gicback/test_x_multi.npy')

test_dataset = VoiceDatasetSimple(X=X_test, y=None, transform=transform, inference=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=30, shuffle=False, num_workers=8)

result=0
for person in range(3):
    for fold in range(5):
        with torch.no_grad():
            weights = torch.load(f'model/network-epoch{epochs}-person({person})-fold({fold}).pth')
            model.load_state_dict(weights['model'])
            model.eval()
            preds = []
            for X in tqdm(test_loader):
                X = torch.tensor(X, dtype=torch.float32, device=device)
                pred = model(X).detach().cpu().numpy().tolist()
                preds+=pred
        preds = np.array(preds)
        result+=preds

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.iloc[:,1:] = result/15
submission.to_csv('submission1.csv', index=False)

ㄴ그냥 sota 모델 dann 쓰기로함

In [ ]:
import os
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


In [ ]:
%cd /content/open

/content/open


In [ ]:
!pip install resampy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import resampy

In [ ]:
train_folder = ""
train_csv = "train.csv"

train_df = pd.read_csv(train_csv)

def extract_features(file_path):
    audio, _ = librosa.load(file_path, res_type="kaiser_fast")
    mfcc = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=40)
    # Pad or truncate the mfcc features to a fixed length
    mfcc = librosa.util.fix_length(mfcc, size=103, axis=1, mode='constant')
    return mfcc

X = np.array([extract_features(os.path.join(train_folder, f)) for f in train_df["path"]])
y = train_df["label"]

# Reshape X to 2D array
X = X.reshape(X.shape[0], -1)

# Normalize the feature data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode the label data
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train.shape
X.shape

(5001, 4120)

In [ ]:
# Determine the number of features
num_features = X_train.shape[1]

# Reshape the input to have a 4D shape
X_train = X_train.reshape(X_train.shape[0], 1, num_features, 1)
X_val = X_val.reshape(X_val.shape[0], 1, num_features, 1)

# Define the input layer
input_layer = Input(shape=(1, num_features, 1))

# Feature extractor
x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_layer)
x = BatchNormalization()(x)

x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)

# Label classifier
label_output = Dense(6, activation="softmax")(x)

# Domain classifier
domain_output = Dense(2, activation="softmax")(x)
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras import layers

import librosa

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import argparse
import wandb
from wandb.keras import WandbCallback

wandb.init(project="DACON_235905", name="Baseline")

parser = argparse.ArgumentParser(description="Baseline")
parser.add_argument('--sampling_rate', default=16000, type=int)
parser.add_argument('--learning_rate', default=0.01, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--cv', default=10, type=int)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

sampling_rate=args.sampling_rate
learning_rate=args.learning_rate
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
cv = args.cv
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

train.head()
model = Model(inputs=input_layer, outputs=[label_output, domain_output])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=["categorical_crossentropy", "categorical_crossentropy"],
              metrics=["accuracy"])

# Train the model
model.fit(X_train, [y_train, domain_labels_train],
          validation_data=(X_val, [y_val, domain_labels_val]),
          batch_size=32, epochs=10,
          callbacks=[EarlyStopping(patience=3)])


ValueError: ignored

In [ ]:
test_folder = ""
test_csv = "test.csv"

test_df = pd.read_csv(test_csv)

# Extract features from test audio samples
X_test = np.array([extract_features(os.path.join(test_folder, f)) for f in test_df["path"]])
X_test = scaler.transform(X_test)

# Make predictions
y_pred = model.predict(X_test)[0]
y_pred_labels = np.argmax(y_pred, axis=1)
predicted_labels = label_encoder.inverse_transform(y_pred_labels)
test_df["label"] = predicted_labels

# Save the predictions to submission file
submission_df = test_df[["id", "label"]]
submission_df.to_csv("submission.csv", index=False)


ValueError: ignored

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=122b9bd46124d763779e7aeba2f7e7b81bb52bf66e98b31ac52e855cafe69d12
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras import layers

import librosa

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import argparse
import wandb
from wandb.keras import WandbCallback

wandb.init(project="DACON_235905", name="Baseline")

parser = argparse.ArgumentParser(description="Baseline")
parser.add_argument('--sampling_rate', default=16000, type=int)
parser.add_argument('--learning_rate', default=0.01, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--cv', default=10, type=int)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

sampling_rate=args.sampling_rate
learning_rate=args.learning_rate
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
cv = args.cv
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 9


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 56


<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

In [ ]:
train_folder = ""
test_folder = "data/test/"

def dataset(folder, mode):
    dataset = []
    for file in tqdm(os.listdir(folder)):
        if 'wav' in file:
            abs_file_path = os.path.join(folder, file)
            data, sr = librosa.load(abs_file_path, sr=sampling_rate)
            data = librosa.util.normalize(data)
            if mode == "train":
                class_label = int(train[train.file_name == file].label)
                dataset.append([data, file, class_label])
            elif mode == "test":
                dataset.append([data, file])
    if mode == "train":
        dataset = pd.DataFrame(dataset, columns=['data', 'file_name', 'label'])
    if mode == "test":
        dataset = pd.DataFrame(dataset, columns=['data', 'file_name'])
    return dataset

train_df = dataset(train_folder, "train")
test_df = dataset(test_folder, "test")

In [ ]:
%cd /content/open

/content/open


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive
import random
import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
sr = 20000


In [ ]:
for i in os.listdir('./train')[:3]:
  print(f'{i}')
  data, sample_rate = librosa.load(f'./train/{i}', sr=sr)
  print('sample_rate:', sample_rate, ', audio shape:', data.shape)
  print('length:', data.shape[0]/float(sample_rate), 'secs')
  print('\n')

FileNotFoundError: ignored

In [ ]:
from tqdm.notebook import tqdm

train_file_names = train["path"].to_numpy()
test_file_names = test["path"].to_numpy()
target = train["label"].to_numpy()

def load_audio(file_names, target, path):
  audios = []
  for audio in tqdm(file_names):
    # librosa를 이용하여 데이터 로드
    an_audio, _ = librosa.load(path+audio, sr=sr)
    audio_array = np.array(an_audio)
    audios.append(audio_array)
  audios = np.array(audios)

  targets = target.copy()

  return audios, targets

In [ ]:
audio_train, target_train = load_audio(train_file_names, target, path='')
audio_test, _ = load_audio(test_file_names, np.array([None]), path='')

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [ ]:
np.save('/content/drive/MyDrive/bbuckkugi/target_train.npy',target_train)
np.save('/content/drive/MyDrive/bbuckkugi/audio_train.npy',audio_train)
np.save('/content/drive/MyDrive/bbuckkugi/audio_test.npy',audio_test)


In [ ]:
def random_pad(mels, pad_size, mfcc=True):

  pad_width = pad_size - mels.shape[1]
  rand = np.random.rand()
  left = int(pad_width * rand)
  right = pad_width - left

  if mfcc:
    mels = np.pad(mels, pad_width=((0,0), (left, right)), mode='constant')
    local_max, local_min = mels.max(), mels.min()
    mels = (mels - local_min)/(local_max - local_min)
  else:
    local_max, local_min = mels.max(), mels.min()
    mels = (mels - local_min)/(local_max - local_min)
    mels = np.pad(mels, pad_width=((0,0), (left, right)), mode='constant')


  return mels

In [ ]:
size = 100
pad_size = 240
repeat_size = 5

In [ ]:
audio_mels = []
audio_mfcc = []

for y in audio_train:
  mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=size)
  mels = librosa.power_to_db(mels, ref=np.max)

  mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=size)

  for i in range(repeat_size):
    audio_mels.append(random_pad(mels, pad_size=pad_size, mfcc=False))
    audio_mfcc.append(random_pad(mfcc, pad_size=pad_size, mfcc=True))

In [ ]:
audio_mels_array_test = []
audio_mfcc_array_test = []

for y in audio_test:
  mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=size)
  mels = librosa.power_to_db(mels, ref=np.max)

  mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=size)

  audio_mels_array_test.append(random_pad(mels, pad_size=pad_size, mfcc=False))
  audio_mfcc_array_test.append(random_pad(mfcc, pad_size=pad_size, mfcc=True))

In [ ]:
audio_mels_array = np.array(audio_mels, np.float64)
audio_mfcc_array = np.array(audio_mfcc, np.float64)

audio_mels_array_test = np.array(audio_mels_array_test, np.float64)
audio_mfcc_array_test = np.array(audio_mfcc_array_test, np.float64)

In [ ]:
from sklearn.model_selection import train_test_split

repeated_target = np.repeat(target_train,repeat_size)


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Conv2D, MaxPool2D, ZeroPadding2D, BatchNormalization, Input, DepthwiseConv2D, Add, LeakyReLU, ReLU
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
def residual_block(x, filters_in, filters_out):
    shortcut = x
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters_in, kernel_size=(1, 1), strides=(1, 1), padding="same",kernel_initializer='he_normal')(x)

    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters_in, kernel_size=(3, 3), strides=(1, 1), padding="same",kernel_initializer='he_normal')(x)

    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1), padding="same",kernel_initializer='he_normal')(x)

    shortcut_channel = x.shape.as_list()[0]

    if shortcut_channel != filters_out:
        shortcut = Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1), padding="same",kernel_initializer='he_normal')(shortcut)

    x = Add()([x, shortcut])
    return ReLU()(x)


In [ ]:
def build_model():

  inputs = tf.keras.layers.Input(shape=(size,pad_size,1))

  outputs = Conv2D(16,(3,3),activation=None,padding='same',kernel_initializer='he_normal')(inputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = MaxPool2D((2,2))(outputs)

  outputs = residual_block(outputs, 16, 32)
  outputs = MaxPool2D((2,2))(outputs)
  outputs = residual_block(outputs, 32, 32)
  # outputs = residual_block(outputs, 32, 32)
  outputs = residual_block(outputs, 32, 64)
  outputs = MaxPool2D((2,2))(outputs)
  outputs = residual_block(outputs, 64, 64)
  # outputs = residual_block(outputs, 64, 64)
  # outputs = MaxPool2D((2,2))(outputs)

  outputs = GlobalAveragePooling2D()(outputs)
  # outputs = Flatten()(outputs)

  outputs = Dense(32,activation=None,kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = Dropout(0.5)(outputs)

  outputs = Dense(6,activation='softmax')(outputs)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

# model = build_model()
# model.summary()

In [ ]:
!pip install h5py
import h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint, EarlyStopping


acc_list = []
pred_list = []
skf = StratifiedKFold(n_splits=5)

for fold,(train_index, val_index) in enumerate(skf.split(audio_mels_array, repeated_target)):

  print(f'\n********** {fold+1} fold **********')

  preds_val_list = []
  ### melspectrogram ###
  model = build_model()
  x_train, x_val, y_train, y_val = audio_mels_array[train_index], audio_mels_array[val_index], repeated_target[train_index], repeated_target[val_index]
  filepath = f"mel_{fold}.hdf5"
  checkpoint = ModelCheckpoint(filepath= filepath, monitor='val_loss', verbose=1, save_best_only=True)
  callbacks=[checkpoint]
  history = model.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_val,y_val), callbacks=callbacks, verbose=1)
  model = load_model(filepath)

  preds_val = model.predict(x_val)
  preds_val_list.append(preds_val)
  preds_val_label = np.argmax(preds_val, axis=1)
  pred_list.append(model.predict(audio_mels_array_test))
  print(f'mels_model_acc : {accuracy_score(y_val,preds_val_label):.4f}')

  ### mfcc ###
  model = build_model()
  x_train, x_val, y_train, y_val = audio_mfcc_array[train_index], audio_mfcc_array[val_index], repeated_target[train_index], repeated_target[val_index]
  filepath = f"mfcc_{fold}.hdf5"
  checkpoint = ModelCheckpoint(filepath= filepath, monitor='val_loss', verbose=1, save_best_only=True)
  callbacks=[checkpoint]
  history = model.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_val,y_val), callbacks=callbacks, verbose=0)
  model = load_model(filepath)

  preds_val = model.predict(x_val)
  preds_val_list.append(preds_val)
  preds_val_label = np.argmax(preds_val, axis=1)
  pred_list.append(model.predict(audio_mfcc_array_test))
  print(f'mfcc_model_acc : {accuracy_score(y_val,preds_val_label):.4f}')

  ### ensemble ###
  val_pred_result = preds_val_list[0].copy()
  for i in range(1, len(preds_val_list)):
      val_pred_result += preds_val_list[i]
  val_pred_label = np.argmax(val_pred_result, axis=1)
  en_acc = accuracy_score(y_val,val_pred_label)
  acc_list.append(en_acc)
  print(f'ensemble_model_acc : {en_acc:.4f}')

print(f'\n\nmean_acc : {np.mean(acc_list):.4f}')


********** 1 fold **********
626/626 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2011
Epoch 1: val_loss did not improve from inf
626/626 [==============================] - 27s 19ms/step - loss: nan - accuracy: 0.2011 - val_loss: nan - val_accuracy: 0.1734


OSError: ignored

In [ ]:
np.save('/content/drive/MyDrive/bbuckkugi/audio_mels_array.npy', audio_mels_array)
np.save('/content/drive/MyDrive/bbuckkugi/repeated_target.npy', repeated_target)
np.save('/content/drive/MyDrive/bbuckkugi/audio_mfcc_array.npy', audio_mfcc_array)
np.save('/content/drive/MyDrive/bbuckkugi/audio_mels_array_test.npy', audio_mels_array_test)
np.save('/content/drive/MyDrive/bbuckkugi/audio_mfcc_array_test.npy', audio_mfcc_array_test)


In [ ]:
type(audio_mels_array)

numpy.ndarray